### Calculating distributions of different properties from Ariane output
This notebook has examples for calculating 1, 2, and 3D distributions from Ariane trajectory output.  
The usage builds on the functions in ariane_distributionfunctions.  
The distributions are saved in `data/processed/lagrangian/`

#### Load the data and define some variables

In [46]:
import numpy as np
import scipy.io as sio
from matplotlib import pyplot as plt
import xarray as xr
from xhistogram.xarray import histogram as hist
from thermodynamics import calc_sigmantr
from ariane_general import ariane_year
from ariane_distributionfunctions import ariane_D1, ariane_D2, ariane_D3, ariane_D4

In [47]:
rootdir = '/home/ocean_personal_data/graemem/ariane/'
model = 'orca025_global_5d'
experiment = 'quant_back_seedNAn1_t*_sign27.7-28_MLrefz8delsig0.01'
filepath = rootdir+'experiments/'+model+'/'+experiment+'/ariane_positions_quantitative.nc'
filepath_initial = rootdir+'experiments/'+model+'/'+experiment+'/ariane_initial.nc'
filepath_time = rootdir+'time/time_orca025_global_5d.mat'
filepath_region = rootdir+'experiments/'+model+'/quant_back_seedNAn1_t3560-sep-4217_sign27.7-28_MLrefz8delsig0.01/region_limits'
# Universal variables
spy = 365*24*60*60
yrst = 1958
yrend = 2016
ventsec = 7
lastinit = 4217

In [48]:
# Ariane input
ds_initial = xr.open_mfdataset(filepath_initial,combine='nested',concat_dim='ntraj')
ds_initial.init_volume.name = 'init_volume'
# Ariane output
ds = xr.open_mfdataset(filepath,combine='nested',concat_dim='ntraj')
ds = xr.merge([ds, ds_initial.init_volume])
ds['final_age'] = ds.final_age.astype('timedelta64[s]').astype('float64')/spy
ds['init_dens'] = calc_sigmantr(ds.init_temp,ds.init_salt)
ds['final_dens'] = calc_sigmantr(ds.final_temp,ds.final_salt)
# Model times
time_vals = np.append(np.array([0]),sio.loadmat(filepath_time)['time'].squeeze())
time = xr.DataArray(time_vals,dims=['nfile'],coords={'nfile':np.arange(time_vals.size)})
# Region limits
region_limits = np.loadtxt(filepath_region)
# Calculate year
ds['final_year'] = ariane_year(ds['final_t'],time,dim='nfile',yrst=1958)

In [49]:
# Bins
years = np.arange(yrst,yrend+1)
ages = np.arange(-3/12,yrend-yrst+9/12)
densities = np.arange(27.7,28,0.01)
init_t_unique = np.unique(ds.init_t)
inits = np.append(init_t_unique-0.5,init_t_unique[-1]+0.5)
xs = np.arange(region_limits[0,0],region_limits[0,1])
ys = np.arange(region_limits[1,0],region_limits[1,1])
lats = np.arange(-30,80,5)

#### A 1-D distribution

In [50]:
var1 = ds.final_age
bins = [ages]
weights = ds.init_volume
S = (ds.final_section==ventsec) & (ds.init_t==lastinit)
subsetname = 'final_section-7_init_t-last'

D1 = ariane_D1(
    var1=var1,
    bins=bins,
    dim=['ntraj'],
    weights=weights,
    subset=True,
    conditional=S,
    subsetname=subsetname,
    save=True
        )

#### A 2-D distribution

In [51]:
var1 = ds.final_age
var2 = ds.init_t
bins = [ages,inits]
weights = ds.init_volume
# weights.name='iViL'
S = (ds.final_section==ventsec)
subsetname = 'final_section-7'

D2 = ariane_D2(
    var1=var1,
    var2=var2,
    bins=bins,
    dim=['ntraj'],
    weights=weights,
    subset=True,
    conditional=S,
    subsetname=subsetname,
    save=True
        )

PermissionError: [Errno 13] Permission denied: b'/home/ocean_personal_data/graemem/ariane/python/draw_figs_nadw_var/data/processed/lagrangian/D2_weights-init_volume_bins-final_age-init_t_subset-final_section-7.nc'

#### A 3-D distribution

In [53]:
var1 = ds.final_age
var2 = ds.final_year
var3 = ds.init_t
bins = [ages,years,inits]
weights = ds.init_volume*ds.init_lat
# weights.name='iViT'
S = (ds.final_section==ventsec)
subsetname = 'final_section-7'

D3 = ariane_D3(
    var1=var1,
    var2=var2,
    var3=var3,
    bins=bins,
    dim=['ntraj'],
    weights=weights,
    subset=True,
    conditional=S,
    subsetname=subsetname,
    verbose=True,
    save=True
        )

init_t : 1367.5 to 1440.5
init_t : 1440.5 to 1513.5
init_t : 1513.5 to 1586.5
init_t : 1586.5 to 1659.5
init_t : 1659.5 to 1732.5
init_t : 1732.5 to 1805.5
init_t : 1805.5 to 1878.5
init_t : 1878.5 to 1952.5
init_t : 1952.5 to 2025.5
init_t : 2025.5 to 2098.5
init_t : 2098.5 to 2171.5
init_t : 2171.5 to 2244.5
init_t : 2244.5 to 2317.5
init_t : 2317.5 to 2390.5
init_t : 2390.5 to 2463.5
init_t : 2463.5 to 2536.5
init_t : 2536.5 to 2609.5
init_t : 2609.5 to 2682.5
init_t : 2682.5 to 2755.5
init_t : 2755.5 to 2828.5
init_t : 2828.5 to 2901.5
init_t : 2901.5 to 2974.5
init_t : 2974.5 to 3047.5
init_t : 3047.5 to 3120.5
init_t : 3120.5 to 3193.5
init_t : 3193.5 to 3266.5
init_t : 3266.5 to 3339.5
init_t : 3339.5 to 3413.5
init_t : 3413.5 to 3486.5
init_t : 3486.5 to 3559.5
init_t : 3559.5 to 3632.5
init_t : 3632.5 to 3705.5
init_t : 3705.5 to 3778.5
init_t : 3778.5 to 3851.5
init_t : 3851.5 to 3924.5
init_t : 3924.5 to 3997.5
init_t : 3997.5 to 4070.5
init_t : 4070.5 to 4143.5
init_t : 414

TypeError: can only concatenate str (not "NoneType") to str

### A 4-D distribution
This could take a considerable amount of time to run. See calc_D4.py for a python script that can be run outside of the notebook environment.

In [ ]:
var1 = ds.final_x
var2 = ds.final_y
var3 = ds.final_age
var4 = ds.init_t

bins = [xs,ys,ages,inits]
weights = ds.init_volume

S = (ds.final_section==ventsec)
subsetname = 'final_section-7'

D4 = ariane_D4(
    var1=var1,
    var2=var2,
    var3=var3,
    var4=var4,
    bins=bins,
    dim=['ntraj'],
    weights=weights,
    subset=True,
    conditional=S,
    subsetname=subsetname,
    verbose=True,
    save=True
        )

init_t : 1367.5 to 1440.5
end
end
init_t : 1440.5 to 1513.5
